如果用户有包含数百万甚至数十亿个节点或边的大图，通常无法直接进行全图训练。  
考虑在一个有 N 个节点的图上运行的、隐层大小为 H 的 L 层图卷积网络， 存储隐层表示需要 O(NLH) 的内存空间，当 N 较大时，这很容易超过一块GPU的显存限制。  
本章介绍了一种在大图上进行**随机小批次训练**的方法，可以让用户不用一次性把所有节点特征拷贝到GPU上。

### **邻居节点采样的工作流程通常如下**：  

每次梯度下降，选择一个小批次的图节点， 其最终表示将在神经网络的第 L 层进行计算，然后在网络的第 L−1 层选择该批次节点的全部或部分邻居节点。  
重复这个过程，直到到达输入层。这个迭代过程会构建计算的依赖关系图，从输出开始，一直到输入

#### 定义邻居采样器和数据加载器

In [ ]:
import dgl
import dgl.nn as dglnn
import torch
import torch.nn as nn
import torch.nn.functional as F

sampler = dgl.dataloading.MultiLayerFullNeighborSampler(2)

#以下代码创建了一个PyTorch的 DataLoader，它分批迭代训练节点ID数组 train_nids， 并将生成的子图列表放到GPU上
dataloader = dgl.dataloading.NodeDataLoader(
        g, train_nids, sampler,
        batch_size=1024,
        shuffle=True,
        drop_last=False,
        num_workers=4)

对DataLoader进行迭代，将会创建一个特定图的列表，这些图表示每层的计算依赖。在DGL中称之为 块。

In [ ]:
input_nodes, output_nodes, blocks = next(iter(dataloader))
print(blocks)

调整模型以进行小批次训练, 用户所需要做的就是用上面生成的块( block )来替换图( g )

In [ ]:
class StochasticTwoLayerGCN(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super().__init__()
        self.conv1 = dgl.nn.GraphConv(in_features, hidden_features)
        self.conv2 = dgl.nn.GraphConv(hidden_features, out_features)

    def forward(self, blocks, x):
        x = F.relu(self.conv1(blocks[0], x))
        x = F.relu(self.conv2(blocks[1], x))
        return x

模型训练

In [ ]:
model = StochasticTwoLayerGCN(in_features, hidden_features, out_features)
model = model.cuda()
opt = torch.optim.Adam(model.parameters())

for input_nodes, output_nodes, blocks in dataloader:
    blocks = [b.to(torch.device('cuda')) for b in blocks]
    input_features = blocks[0].srcdata['features']
    output_labels = blocks[-1].dstdata['label']
    output_predictions = model(blocks, input_features)
    loss = compute_loss(output_labels, output_predictions)
    opt.zero_grad()
    loss.backward()
    opt.step()